In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# %load ../../loaders/imports.py
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import time
import pdb

# Add the uoicorr directory to the path
sys.path.append('../../../uoicorr')

# Add the root directory of this repository
sys.path.append('../..')

from utils import gen_covariance, gen_beta2, gen_data, get_cov_list
from utils import selection_accuracy
from sklearn.linear_model import LassoLars, lasso_path, LinearRegression

from pyuoi.linear_model import UoI_Lasso

In [6]:
from LLA.lla import LLA
from LLA.lla import dSCAD, dMCP
from LLA.lla import FISTA

In [4]:
from sklearn.preprocessing import StandardScaler

In [5]:
# %load ../../loaders/datgen.py
n_features = 50
n_samples = 150

sigma = gen_covariance(n_features, 0, n_features, 1, 0)
beta = gen_beta2(n_features, n_features, sparsity = 0.2, betawidth = np.inf)
X, X_test, y, y_test, ss = gen_data(n_samples, n_features, kappa = 100, 
									covariance = sigma, beta = beta)
X = StandardScaler().fit_transform(X)
y -= np.mean(y)

In [9]:
# Initialize with Lasso
from sklearn.linear_model import Lasso

In [ ]:
l = Lasso(fit_intercept=False, alpha = 1)

In [53]:
%timeit l.fit(X, y)

244 µs ± 12.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [33]:
l.coef_

array([ 0.        ,  0.        , -0.        , -0.        ,  0.        ,
       -0.        ,  6.88007303, -0.        ,  0.        ,  8.2038499 ,
        0.        ,  8.91877782, -0.        ,  3.62031202,  0.        ,
       -0.        , -0.        , -0.        ,  0.        ,  0.        ,
       -0.        , -0.        ,  3.99104761, -0.        ,  5.62299789,
        2.25769429, -0.        ,  0.        ,  0.        , -0.        ,
       -0.        ,  0.        , -0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.        ,  4.07008385, -0.        , -0.        ,
        0.        , -0.        ,  0.        ,  0.        ,  4.6829654 ])

In [55]:
# Test whether the FISTA algorithm gives good solutions to the Lasso
%timeit coefs = FISTA(np.zeros(n_features), X, y.ravel(), np.ones(n_features), max_iter=2000)

1.43 ms ± 22.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [49]:
coefs

(array([ 0.        ,  0.        , -0.        , -0.        ,  0.        ,
        -0.        ,  6.88008002, -0.        ,  0.        ,  8.20372679,
         0.        ,  8.92006094, -0.        ,  3.62028592,  0.        ,
        -0.        , -0.        , -0.        ,  0.        ,  0.        ,
        -0.        , -0.        ,  3.99331747, -0.        ,  5.62351295,
         2.25541176, -0.        ,  0.        ,  0.        , -0.        ,
        -0.        ,  0.        , -0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -0.        ,  4.06947193, -0.        , -0.        ,
         0.        , -0.        ,  0.        ,  0.        ,  4.68497351]),
 54.61932325012915,
 8.579585468737605e-07,
 37)

In [61]:
beta = LLA(np.zeros(n_features), X, y.ravel(), dSCAD, (3, 1))

In [65]:
len(beta)

2

#### Pycasso solver

In [56]:
import pycasso

In [57]:
solver = pycasso.Solver(X, y, family='gaussian', useintercept=False, gamma=3, penalty = 'scad', lambdas = (1, 0.5, 0.25))

In [58]:
solver.train()

In [59]:
solver.result['beta']

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         8.16006833e+00,  0.00000000e+00,  0.00000000e+00,
         8.88281286e+00,  0.00000000e+00,  9.47446164e+00,
         0.00000000e+00,  4.40699424e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  5.13220065e+00,  0.00000000e+00,
         6.27599722e+00,  3.12171391e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         4.91220639e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  5.77869005e+00],
       [ 0.00

[array([-0.        , -0.        , -0.        , -0.        ,  0.        ,
        -0.        ,  8.15897958,  0.        ,  0.        ,  8.88319147,
         0.        ,  9.47755123, -0.        ,  4.40555114, -0.        ,
        -0.        , -0.        , -0.        , -0.        , -0.        ,
         0.        ,  0.        ,  5.13116975,  0.        ,  6.27514289,
         3.1213641 , -0.        , -0.        , -0.        , -0.        ,
        -0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.        ,  0.        , -0.        ,  0.        ,  0.        ,
         0.        , -0.        ,  4.91260809,  0.        , -0.        ,
         0.        , -0.        ,  0.        , -0.        ,  5.77854679]),
 array([-0.        , -0.        , -0.        , -0.        ,  0.        ,
        -0.        ,  8.15897996,  0.        ,  0.        ,  8.88319136,
         0.        ,  9.47755102, -0.        ,  4.40555117, -0.        ,
        -0.        , -0.        , -0.        , -0

In [63]:
def SCAD_loss(X, y, beta, alpha, gamma):
    MSE = 1/(2 * n_samples) * np.linalg.norm(y - X @ beta)**2
    penalty = np.sum([alpha * np.abs(xi) if np.abs(xi) <= alpha
                      else (2 * gamma * alpha * np.abs(xi) - xi**2 - alpha**2)/(2 * (gamma - 1)) 
                      if np.abs(xi) > alpha and np.abs(xi) < alpha * gamma
                      else alpha**2 * (gamma + 1)/2 for xi in beta])
    
    return MSE + penalty
    

In [69]:
def MCP_loss(X, y, beta, alpha, gamma): 
    
    MSE = 1/(2 * n_samples) * np.linalg.norm(y - X @ beta)**2
    penalty = np.sum([alpha * np.abs(xi) - xi**2/(2 * gamma) if np.abs(xi) <= gamma * alpha
                      else 1/2 * gamma * alpha**2 for xi in beta])
    return MSE + penalty

In [64]:
SCAD_loss(X, y, solver.result['beta'][0, :], 1, 3)

70964.58975323693

In [68]:
SCAD_loss(X, y, beta[1], 1, 3)

70966.64522035174

In [ ]:
# A few questions: 
# 1) Verify superiority of Pycasso to LLA using (1) 0 initialization across a few different designs
#                                               (2) Lasso initialization
#                                               (3) SCAD/MCP initialization

In [ ]:
# Next up: Debug empirical bayes and the use of aBIC across UoI with the stability selection knob turned down

In [71]:
# Choose a block diagonal, exponential, uncorrelated design matrix, as well as equal and uniformly distributed
# coefficients. Then, do 5 repetitions. Record the selection accuracy, SCAD/MCP loss for all
n_features = 200
n_samples = 800

cov_params = [{'correlation' : 0.25, 'block_size' : 25, 'L' : 1, 't' : 0},
              {'correlation' : 0, 'block_size' : 200, 'L' : 1, 't' : 0},
              {'correlation' : 0.25, 'block_size' : 200, 'L' : 10, 't' : 1}]

sparsity = 0.25
betawidth = [0.1, np.inf]
kappa = [1, 2, 5]
reps = 5

# 3 different initializations
LLA_SCAD_loss = np.zeros((len(cov_params), len(betawidth), len(kappa), reps, 2))
LLA_MCP_loss = np.zeros((len(cov_params), len(betawidth), len(kappa), reps, 2))
pyc_SCAD_loss = np.zeros((len(cov_params), len(betawidth), len(kappa), reps))
pyc_MCP_loss = np.zeros((len(cov_params), len(betawidth), len(kappa), reps))

LLA_SCAD_sa = np.zeros((len(cov_params), len(betawidth), len(kappa), reps, 2))
LLA_MCP_sa = np.zeros((len(cov_params), len(betawidth), len(kappa), reps, 2))
pyc_SCAD_sa = np.zeros((len(cov_params), len(betawidth), len(kappa), reps))
pyc_MCP_sa = np.zeros((len(cov_params), len(betawidth), len(kappa), reps))

for i1, cov_param in enumerate(cov_params):
    for i2, bw in enumerate(betawidth): 
        for i3, k in enumerate(kappa):
            for rep in range(reps): 
                
                # Generate data
                sigma = gen_covariance(n_features, **cov_param)
                beta = gen_beta2(n_features, cov_param['block_size'], sparsity = sparsity,
                                 betawidth = bw)
                X, X_test, y, y_test, ss = gen_data(n_samples, n_features, kappa = k, 
                                                    covariance = sigma, beta = beta)
                X = StandardScaler().fit_transform(X)
                y -= np.mean(y)
                
                ## SCAD ##
                solver = pycasso.Solver(X, y, family='gaussian', useintercept=False, gamma=3,
                                        penalty = 'scad', lambdas = (1, 0.5, 0.25))

                pyc_coefs = solver.result['beta'][0, :]

                pyc_SCAD_loss[i1, i2, i3, rep] = SCAD_loss(X, y.ravel(), pyc_coefs.ravel(), 1, 3)
                pyc_SCAD_sa[i1, i2, i3, rep] = selection_accuracy(beta.ravel(), pyc_coefs.ravel())

                # Zero initialization
                lla_coefs = LLA(np.zeros(n_features), X, y.ravel(), dSCAD, (3, 1))[0]
                
                LLA_SCAD_loss[i1, i2, i3, rep, 0] = SCAD_loss(X, y.ravel(), lla_coefs.ravel(), 1, 3)
                LLA_SCAD_sa[i1, i2, i3, rep, 0] = selection_accuracy(beta.ravel(), lla_coefs.ravel())
                
                # SCAD initialization
                lla_coefs = LLA(pyc_coefs.ravel(), X, y.ravel(), dSCAD, (3, 1))[0]
                LLA_SCAD_loss[i1, i2, i3, rep, 1] = SCAD_loss(X, y.ravel(), lla_coefs.ravel(), 1, 3)
                LLA_SCAD_sa[i1, i2, i3, rep, 1] = selection_accuracy(beta.ravel(), lla_coefs.ravel())
                
                
                ## MCP ## 
                solver = pycasso.Solver(X, y, family='gaussian', useintercept=False, gamma=3,
                                        penalty = 'mcp', lambdas = (1, 0.5, 0.25))
                
                pyc_coefs = solver.result['beta'][0, :]
                
                pyc_MCP_loss[i1, i2, i3, rep] = MCP_loss(X, y.ravel(), pyc_coefs.ravel(), 1, 3)
                pyc_MCP_sa[i1, i2, i3, rep] = selection_accuracy(beta.ravel(), pyc_coefs.ravel())

                
                # Zero initialization
                lla_coefs = LLA(np.zeros(n_features), X, y.ravel(), dMCP, (3, 1))[0]
                
                LLA_MCP_loss[i1, i2, i3, rep, 0] = MCP_loss(X, y.ravel(), lla_coefs.ravel(), 1, 3)
                LLA_MCP_sa[i1, i2, i3, rep, 0] = selection_accuracy(beta.ravel(), lla_coefs.ravel())
                
                # SCAD initialization
                lla_coefs = LLA(pyc_coefs.ravel(), X, y.ravel(), dMCP, (3, 1))[0]
                LLA_MCP_loss[i1, i2, i3, rep, 1] = MCP_loss(X, y.ravel(), lla_coefs.ravel(), 1, 3)
                LLA_MCP_sa[i1, i2, i3, rep, 1] = selection_accuracy(beta.ravel(), lla_coefs.ravel())
                
                

In [73]:
# Achieved loss, 0 initialization
np.where(LLA_MCP_loss[..., 0] < pyc_MCP_loss)

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1]),
 array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2,
        2, 2]),
 array([0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3,

In [79]:
LLA_SCAD_loss[..., 0] < pyc_SCAD_loss

array([[[[ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]],

        [[ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]]],


       [[[ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]],

        [[ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]]],


       [[[ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]],

        [[ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]]]])

In [80]:
LLA_SCAD_sa[..., 0] > pyc_SCAD_sa

array([[[[ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]],

        [[ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]]],


       [[[ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]],

        [[ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]]],


       [[[ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]],

        [[ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]]]])